In [5]:
%matplotlib inline
import urllib
import matplotlib.pyplot as pp
import pylab
import numpy as np
from pycbc.waveform import get_td_waveform
from pycbc.filter.matchedfilter import sigmasq
from pycbc.detector import Detector
from pycbc.psd import analytical
from pycbc.psd import read as psd_read    

def calculate_snr(mass, d):
    ifos=['H1', 'L1', 'V1'] 
    sample_rate=1024

    # Create n waveforms:
    n=10
    apx='IMRPhenomD'
    distance=400 

    # masses= np.random.randint(10, 35, size=n)
    ra=2.2
    dec=-1.25
    pol=2.1 
    t_gps=120
    
    hp, hc=get_td_waveform(approximant=apx,
                             mass1=mass,
                             mass2=mass,
                             delta_t=1.0/sample_rate,
                             f_lower=10,
                             distance=d)
    flow = 10.0
    delta_f = 1.0 / hp.duration 
    flen = int(sample_rate/delta_f)
#     psd = analytical.aLIGOZeroDetHighPower(flen, delta_f, flow) #optimize by calculating only once
    read_data = np.genfromtxt('o3_h1.txt')
    psd_data = read_data[:, 1]
    psd_data = psd_data ** 2
    freq = read_data[:, 0]
    psd = psd_read.from_numpy_arrays(freq, psd_data,flen, delta_f, 20)
    psd.data[np.argwhere(psd.data == 0)] = 1
    
    gw = {'hp': hp, 'hc':hc, 'psd': psd}
    
    for ifo in ifos:
        det=Detector(ifo)
        fp, fc=det.antenna_pattern(ra, dec, pol, t_gps)
    
        s= fp * gw['hp'] + fc * gw['hc']
        snr = sigmasq(s,psd=gw['psd'], low_frequency_cutoff=10)
        if 'snr_sq' in gw:  
            gw['snr_sq'] += snr
        else:
            gw['snr_sq'] = snr
            
    return gw['snr_sq']


mass 17 = 38.62607133668334 snr
mass 27 = 56.0377164472265 snr
mass 20 = 44.055492844356735 snr
mass 16 = 36.77058263570758 snr
mass 28 = 57.71638848019459 snr


In [ ]:
distance = 1
masses = np.arange(10, 40, 1)
snr_ls = []
max_distance = []

for mass in masses:
    while True:
        snr = calculate_snr(mass, distance)
        snr_ls.append(snr)
        distance += 50
    
        if snr < 8:
            max_distance.append(distance)
            break

pylab.plot(masses, max_distance)
pylab.ylabel('Distance (Mpc)')
pylab.xlabel('Mass (solar mass)')